In [65]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import mne

filename = "X_train/X_train.h5"

eggs = []
for i in range(1, 8):
    np_array = np.array(h5py.File(filename, mode='r')['eeg_'+str(i)])
    eggs.append(np_array)

In [66]:
filename='y_train.csv'

ytrain=np.array(pd.read_csv(filename))
for i in range(5):
    print('occurence sleep stage',i,np.count_nonzero(ytrain[:,1]== i))

occurence sleep stage 0 3511
occurence sleep stage 1 1671
occurence sleep stage 2 9449
occurence sleep stage 3 5224
occurence sleep stage 4 4833


In [67]:
def extractfbands(eeg,cuts):
    L=list()
    s=eeg.shape
    idcut=int(s[1]/cuts)
    freq=np.fft.fftfreq(idcut,1/50)[:int(s[1]/2)]
    print(freq.shape)
    for i in range(cuts):
        print(i)
        FFT=np.fft.fft(eeg[:,i*idcut:(i+1)*idcut],axis=1)[:,:int(s[1]/2)]
        FFT=abs(FFT)**2
        delta=np.sum(FFT[:,np.where((freq<4)*(freq>0))],axis=2)
        theta=np.sum(FFT[:,np.where((freq>4)* (freq<8))],axis=2)
        alpha=np.sum(FFT[:,np.where((freq>8)* (freq<13))],axis=2)
        beta=np.sum(FFT[:,np.where((freq>13 )* (freq<22))],axis=2)
        gamma=np.sum(FFT[:,np.where(freq>22)],axis=2)
        L.append( np.array([delta,theta,alpha,beta,gamma]))
    return np.array(L)


    

In [68]:
eeg1band=extractfbands(eggs[0],3)


(500,)
0
1
2


In [71]:
L=list()
for i in range(7):
    L.append(extractfbands(eggs[i],3))
L=np.array(L)

(500,)
0
1
2
(500,)
0
1
2
(500,)
0
1
2
(500,)
0
1
2
(500,)
0
1
2
(500,)
0
1
2
(500,)
0
1
2


In [72]:
L.shape

(7, 3, 5, 24688, 1)

In [73]:
L2=np.reshape(L,(7,3,5,24688))
print(L2.shape)
rearranged_L = np.moveaxis(L2,[3],[0])
print(rearranged_L.shape)


(7, 3, 5, 24688)
(24688, 7, 3, 5)


In [74]:
final_L=np.reshape(rearranged_L,(24688,7,15))
final_L=np.reshape(final_L,(24688,7*15))

In [76]:
np.save("foo.npy", np.array(final_L))